In [15]:
import requests
import os
from dotenv import load_dotenv, find_dotenv
import pandas as pd


In [16]:
base_url = "https://api.themoviedb.org/3"

In [17]:
page = 1
all_movies = []
load_dotenv()
api_key = os.getenv("API_KEY")

while True:
    url = f"{base_url}/movie/popular?api_key={api_key}&page={page}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        movies = data.get('results', [])

        if not movies:
            break
        
        all_movies.extend(movies)
        page += 1
    else:
        break


In [18]:
#control variables - to be called if error to check status
max_pages_from = (f"Error: Unable to retrieve movies from page {page}")
movies_retrieved = (f"Total movies retrieved: {len(all_movies)}")

In [19]:
all_movies_df = pd.DataFrame(all_movies)

DATA FRAME CLEAN

In [20]:
all_movies_df = pd.DataFrame(all_movies)
# Delete: adult, popularity, video, & vote count
no_use_columns = ['adult', 'backdrop_path', 'popularity', 'video', 'vote_count' ]
# we delete
all_movies_df = all_movies_df.drop(no_use_columns, axis=1)
# convert genres id column to string
all_movies_df['genre_ids'] = all_movies_df['genre_ids'].astype(str)
# clean the strings
all_movies_df['genre_ids'] = all_movies_df['genre_ids'].str.replace('[', '').str.replace(']', '')
# division of column genres id by coma
all_movies_df['genre_ids'] = all_movies_df['genre_ids'].str.split(',').str[0]


/var/folders/n7/qn8s75510p5chs7mw2tvcgy80000gp/T/ipykernel_6689/467950752.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  all_movies_df['genre_ids'] = all_movies_df['genre_ids'].str.replace('[', '').str.replace(']', '')


In [21]:
#from API
genre_mapping = {
    28: "Action",
    12: "Adventure",
    16: "Animation",
    35: "Comedy",
    80: "Crime",
    99: "Documentary",
    18: "Drama",
    10751: "Family",
    14: "Fantasy",
    36: "History",
    27: "Horror",
    10402: "Music",
    9648: "Mystery",
    10749: "Romance",
    878: "Science Fiction",
    10770: "TV Movie",
    53: "Thriller",
    10752: "War",
    37: "Western"
}


In [22]:
# Replace the numbers in the 'genre_ids' column with the corresponding wordsdef map_genre_ids(genre_ids):
def map_genre_ids(genre_ids):
    split_ids = genre_ids.split(',')
    mapped_genres = [genre_mapping.get(int(val.strip()), val.strip()) for val in split_ids if val.strip()]
    return ', '.join(mapped_genres)

all_movies_df['genre_ids'] = all_movies_df['genre_ids'].apply(map_genre_ids)

In [23]:

genre_mapping = {
    "Action",
    "Adventure",
    "Animation",
    "Comedy",
    "Crime",
    "Documentary",
    "Drama",
    "Family",
    "Fantasy",
    "History",
    "Horror",
    "Music",
    "Mystery",
    "Romance",
    "Science Fiction",
    "TV Movie",
    "Thriller",
    "War",
    "Western"
}

In [24]:
#Make sure 'vote_average' is numeric
all_movies_df['vote_average'] = pd.to_numeric(all_movies_df['vote_average'], errors='coerce')

# Create separate DataFrames for each gender and select the 250 highest 'vote_average' values
genre_dataframes = {}
for genre in genre_mapping:
    genre_df = all_movies_df[all_movies_df['genre_ids'].str.contains(genre)]
    genre_df = genre_df.nlargest(250, 'vote_average')
    genre_dataframes[genre] = genre_df

# Concatenate the DataFrames of all genres into one
top_250_movies_by_genre = pd.concat(genre_dataframes.values(), ignore_index=True)

top_250_movies_by_genre = top_250_movies_by_genre.applymap(lambda x: x.lower() if isinstance(x, str) else x)

In [25]:
selected_columns = ["title", "overview", "vote_average", "genre_ids"]
top_250_movies_by_genre = top_250_movies_by_genre[selected_columns]
top_250_movies_by_genre.rename(columns={"genre_ids": "genre"}, inplace=True)
top_250_movies_by_genre.rename(columns={"vote_average": "rating"}, inplace=True)

In [26]:
file_path = 'top_250_movies_by_genre.csv'

top_250_movies_by_genre.to_csv(file_path, index=False) 